In [1]:
import numpy as np
import pandas as pd
from comet_ml import Experiment
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
import numpy as np
from format_data import *

# Entraînement

### Importation et formattage des données

In [2]:
train_data = pd.read_csv("./data/GAN_train.csv", index_col=0)
train_data = format_data(train_data)
train_data

,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,T200,T500,...,LOC_1,LOC_2,LOC_3,LOC_4,LOC_5,SEASON_Fall,SEASON_Spring,SEASON_Summer,SEASON_Winter,Label
0,14.278533,-3.173287,1.833835,-7.581432,3.399386,0.006412,103070.359400,103070.359400,214.141617,256.000519,...,0,0,0,0,0,True,False,False,False,0
1,44.269070,-3.000973,0.375796,-2.194901,-3.694045,0.017199,101155.218800,101155.218800,219.897949,266.757843,...,0,0,0,1,0,True,False,False,False,0
2,29.716904,5.699519,-4.106040,3.079108,-2.678627,0.012480,101841.164100,101841.164100,215.238892,264.979614,...,0,1,0,0,0,True,False,False,False,2
3,36.301456,-0.657589,-1.584453,-0.903698,-2.157597,0.017055,101450.968800,101450.968800,217.602081,267.198273,...,0,0,1,0,0,False,False,True,False,0
4,29.445889,22.721797,-15.070397,1.895406,-7.071098,0.008437,101568.182752,101137.479833,215.028439,265.348473,...,0,0,0,0,0,False,False,True,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21333,16.775038,-4.126678,6.831597,-4.234844,5.019883,0.008671,101950.937500,101950.937500,214.138534,262.182343,...,0,1,0,0,0,False,False,True,False,0
21334,44.922436,-9.911572,0.828655,-5.677947,-1.543543,0.019176,101564.820300,101565.976600,215.284180,265.710754,...,0,0,1,0,0,False,False,True,False,0
21335,22.976133,8.799085,-1.685878,1.721771,-2.926032,0.009924,102670.101600,102670.101600,210.934235,257.265045,...,0,0,0,0,0,False,False,True,False,0
21336,50.522873,0.861236,5.695543,-1.403138,2.239701,0.019056,101072.312500,101072.312500,219.541916,268.598907,...,0,0,0,0,1,False,False,True,False,0


### Division du jeu de données en jeu d'entraînement et de validation

In [3]:
y = train_data["Label"]
X = train_data.drop(columns=["Label"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### Calcul des poids à donner à chacune des classes

In [4]:
class_weights = compute_class_weight('balanced', classes=[0, 1, 2], y=y_train)
weights = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2]}

train_dmatrix = xgb.DMatrix(X_train, label=y_train, weight=y_train.map(weights))
val_dmatrix = xgb.DMatrix(X_val, label=y_val)

c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

### Entraînement

In [5]:
from xgboost.callback import TrainingCallback

class CustomLearningRateCallback(TrainingCallback):
    def __init__(self):
        super().__init__()

    def before_iteration(self, model, epoch, evals_log):
        if epoch < 100:
            model.set_param('eta', 0.1)
        elif epoch < 1000:
            model.set_param('eta', 0.05)
        else:
            model.set_param('eta', 0.01)
        return False

In [9]:
from comet_ml.config import set_global_experiment

hyperparameter = {
    'objective': 'multi:softprob',
    'num_class': 3,
    'booster': 'gbtree',
    'eval_metric': 'merror',
    'eta': 0.01,
    'max_depth': 8,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'alpha': 0.01,
    'seed': 42,
    'num_round': 1000000
}

watchlist = [(train_dmatrix, 'train'), (val_dmatrix, 'validation')]

xgb_model = xgb.train(
hyperparameter, 
train_dmatrix, 
num_boost_round=1000000, 
evals=watchlist, 
early_stopping_rounds=75, 
verbose_eval=True
)

[0]	train-merror:0.06488	validation-merror:0.11215


[1]	train-merror:0.05397	validation-merror:0.09841
[2]	train-merror:0.04843	validation-merror:0.08466
[3]	train-merror:0.04834	validation-merror:0.08997
[4]	train-merror:0.04900	validation-merror:0.08825


c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\core.py:160: UserWarning: [04:07:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)


[5]	train-merror:0.04830	validation-merror:0.08747
[6]	train-merror:0.04846	validation-merror:0.08685
[7]	train-merror:0.04389	validation-merror:0.08138
[8]	train-merror:0.04662	validation-merror:0.08482
[9]	train-merror:0.04520	validation-merror:0.08497
[10]	train-merror:0.04382	validation-merror:0.08201
[11]	train-merror:0.04301	validation-merror:0.08310
[12]	train-merror:0.04441	validation-merror:0.08404
[13]	train-merror:0.04384	validation-merror:0.08247
[14]	train-merror:0.04415	validation-merror:0.08372
[15]	train-merror:0.04447	validation-merror:0.08201
[16]	train-merror:0.04420	validation-merror:0.08169
[17]	train-merror:0.04376	validation-merror:0.08060
[18]	train-merror:0.04414	validation-merror:0.07810
[19]	train-merror:0.04310	validation-merror:0.07763
[20]	train-merror:0.04444	validation-merror:0.07826
[21]	train-merror:0.04500	validation-merror:0.07951
[22]	train-merror:0.04459	validation-merror:0.07888
[23]	train-merror:0.04449	validation-merror:0.07982
[24]	train-merror

In [10]:
y_pred = xgb_model.predict(val_dmatrix)
y_pred = [np.argmax(value) for value in y_pred]

accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Accuracy: 99.30%


# Calcul des prédictions sur le jeu de données test

In [22]:
import pandas as pd

# Assuming train_data has been previously loaded
test_data = pd.read_csv("./data/test.csv")

# Format the test_data
test_data = format_data(test_data, is_test=True)

# List of columns from train_data excluding the label
columns_without_label = [col for col in train_data.columns if col != "Label"]

# Check and add missing columns to test_data and set their values to 0
missing_columns = ['SEASON_Spring', 'SEASON_Winter']

for col in missing_columns:
    if col not in test_data.columns:
        test_data[col] = 0

# Reorder the columns of test_data to match the order in train_data
test_data = test_data[columns_without_label]

test_data

,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,T200,T500,...,LOC_0,LOC_1,LOC_2,LOC_3,LOC_4,LOC_5,SEASON_Fall,SEASON_Spring,SEASON_Summer,SEASON_Winter
0,25.907482,6.662070,-17.510447,-7.432653,-3.936030,0.010624,101532.5391,101532.5391,213.092209,256.032043,...,1,0,0,0,0,0,True,0,False,0
1,25.907482,6.662070,-17.510447,-7.432653,-3.936030,0.010624,101532.5391,101532.5391,213.092209,256.032043,...,1,0,0,0,0,0,True,0,False,0
2,27.019733,4.951319,-17.341263,-7.286631,-3.150316,0.010890,101513.0234,101513.0234,213.161011,255.616837,...,1,0,0,0,0,0,True,0,False,0
3,27.019733,4.951319,-17.341263,-7.286631,-3.150316,0.010890,101513.0234,101513.0234,213.161011,255.616837,...,1,0,0,0,0,0,True,0,False,0
4,26.516499,5.362008,-17.227922,-7.257047,-2.907396,0.010821,101505.1484,101505.1484,213.188248,255.498810,...,1,0,0,0,0,0,True,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,57.411018,3.704696,17.998688,-8.723821,14.131296,0.020041,100295.9375,100295.9375,222.533188,267.842407,...,0,0,0,1,0,0,True,0,False,0
10316,57.277252,5.944778,18.429197,-8.424341,15.523009,0.020222,100210.2891,100210.2891,221.928162,268.028778,...,0,0,0,1,0,0,True,0,False,0
10317,57.277252,5.944778,18.429197,-8.424341,15.523009,0.020222,100210.2891,100210.2891,221.928162,268.028778,...,0,0,0,1,0,0,True,0,False,0
10318,54.855862,8.170049,19.165062,-6.791406,16.326492,0.020324,100116.5234,100116.5234,220.904862,267.992645,...,0,0,0,1,0,0,True,0,False,0


In [23]:
dtest = xgb.DMatrix(test_data)
y_test = xgb_model.predict(dtest)
y_test = [np.argmax(value) for value in y_test]

c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [24]:
labels, counts = np.unique(y_test, return_counts=True)

for label, count in zip(labels, counts):
    print(f"Label {label}: {count} occurrences")

Label 0: 7799 occurrences
Label 1: 601 occurrences
Label 2: 1920 occurrences


In [25]:
df = pd.DataFrame({
    'SNo': range(1, len(y_test) + 1),
    'Label': y_test
})

df.to_csv("prev.csv", index=False)